# Joint Intent detection and slot filling
Dieses Jupyter notebook wurde als semesterabschließende Arbeit für das Modul Natural Language Processing an der [Fachhochschule Südwestfalen](https://www.fh-swf.de/en/international_3/index.php) erstellt.

## Einleitung
Das Joint intent detection and slot filling (IDSF) ist eine Aufgabe aus dem Teilbereich des Natural Language Understandings (NLU) des Natural Language Processings (NLP), die uns in heutzutage fast täglich Alltag begegnet. Sei es um einen Timer auf dem Handy zu starten, bestimmte Musik abzuspielen oder das Licht einzuschalten. Der Ablauf ist dabei häufig der selbe: "Siri stelle einen Timer für 4 Minuten", "Alexa spiele meine Schlager Playlist" oder "Google erstelle einen Arzttermin für heute 16:00 Uhr". Meist beginnen die Kommandos mit dem Namen des Sprachassistenten, um diesen zu aktivieren, gefolgt vom Kommando für die gewünschte Aktion. Das IDSF beschäftigt sich dabei mit der Aufgabe, die gewünschte Aktion (Intent), also stelle einen Timer, Spiele Musik, erstelle einen Termin im Kalender und die dazugehörigen notwendigen Parameter, wie z.B. vier Minuten, Schlager Playlist oder Arzt heute 16:00 Uhr (Slots) zu erkennen.
Da der Gebrauch dieser Sprachassistenten in Zukunft wahrscheinlich noch stärker zu nehmen wird, wollen wir uns deren funktionsweise in diesem Notebook näher anschauen. Dafür wird zuerst die Entwicklungshistorie vom IDSF betrachtet und anschließend wird ein eigenes Modell für die Erkennung erstellt und anhand eines selbst vorbereiteten Korpus trainiert.

## Joint Intent Detection and Slot filling

--------------

## Datenbeschaffung

Als Datensatz für das nachfolgende Beispiel verwenden wir den Snips-Datensatz. Dieser Datensatz wurde vom, mittlerweil zu Sonos gehörenden [1], [Snips Team](https://snips.ai/) zusammengestellt, um ihr eigenes Modell mit anderen Wettbewerbern wie zum Beispiel Amazons Alexa zu verglichen. Die Ergebnisse und die Datensätze der drei Vergleiche wurden in einem [GitHub Repository](https://github.com/sonos/nlu-benchmark/tree/master) veröffentlicht und in dem Paper "Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces" [2] erläutert. Das Repository enthält die Daten für drei Evaluationen aus den Jahren 2016 bis 2018. Wir werden in diesem Notebook die Daten der 2017 durchgeführten Evaluation verwenden, da diese Sätze für sieben unterschiedliche und allgemeine Aufgaben enthält.

Die Daten sind im dem Repository in einzelnen JSON-Dateien enthalten. Dabei gibt es pro Aufgabe zwei Dateien, eine für das Training und eine für die Validierung. Der Einfachheit halber wurden die Dateien in dem data Verzeichnis, dass diesem Notebook beiliegt, abgelegt.

Nachfolgend ist ein Auszug aus der `train_AddToPlaylist_full.json`-Datei.

In [15]:
!head -n 48 data/train_AddToPlaylist_full.json

{
  "AddToPlaylist": [
    {
      "data": [
        {
          "text": "Add another "
        },
        {
          "text": "song",
          "entity": "music_item"
        },
        {
          "text": " to the "
        },
        {
          "text": "Cita Romántica",
          "entity": "playlist"
        },
        {
          "text": " playlist. "
        }
      ]
    },
    {
      "data": [
        {
          "text": "add "
        },
        {
          "text": "clem burke",
          "entity": "artist"
        },
        {
          "text": " in "
        },
        {
          "text": "my",
          "entity": "playlist_owner"
        },
        {
          "text": " playlist "
        },
        {
          "text": "Pre-Party R&B Jams",
          "entity": "playlist"
        }
      ]
    },


Die Datei besteht an oberster Stelle aus dem Namen der gewünschten Aktion gefolgt von einer Liste an Objekten mit einem `Data` Attribut. Dieses enthält wiederum eine Liste von Objekten mit `Text` Attributen die den Satz in Teilen enthält. Dabei wird der Satz durch den Text eines definierten `Entities` geteilt. So enthält das erste Beispiel den Text bis zum ersten `entity` das als `music_item` klassifiziert wurde und wieder den gesamten Text bis zum nächsten entity, dem Namen einer Playlist.

Als nächstes werden die Daten in ein verwendbares Format transformiert. Ein in der Natural language processing gängiges Format ist das IOB Format. IOB steht für Inside-Outside-Beginning. Dieses Format ermöglicht die Kennzeichnung der einzelnen Entitäten in einem Satz. Es wird unteranderem von den weit verbreiteten Python Bibliotheken `NLTK` und `spaCy` unterstützt [3, 4]. Das Format wurde 1995 von Lance A. Ramshaw und Mitchell P. Marcus erfunden.

Dieses Beispiel zeigt das Format einer Zeile, welches nachfolgend aus den JSON-Dateien erzeugt wird.

    BOS add clem burke in my playlist Pre-Party R&B Jams EOS o o b-mucic_item i-music_item o i-playlist_owner o b-playlist i-playlist i-playlist
    
Am Beginn der Zeile steht der vollständige Satz abgetrennt durch ein BOS (begin of sentence) am Anfang des Satzes und ein EOS (end of sentence) am Ende des Satzes. Nun folgt das eigentliche IOB-Format. Dabei wird für jeden Token entweder der Buchstabe 'o', dieser steht für keine Bedeutung, der Buchstabe 'b', für den Beginn einer Entität die aus mehreren Token besteht, oder 'i', als Entität. Das 'i' steht dabei entweder nach einem 'b' wodurch eine Entität gekennzeichent wird, die aus mehreren Token besteht oder alleine für eine Entität die aus nur einem Token besteht.  Das 'b' und 'i' werden dabei jeweils gefolgt vom einem trennenden Bindestrich und der Entitätskategorie verwender. So ist der Name 'Clem Burke' unterteilt in ein `b-music_item` für Clem und `i-Music_item` für Burke. Dadurch wird definiert, dass die beiden Teile zusammen gehören.

Das IOB2 Format ist eine Erweiterung des originalen IOB Formats. Es definiert das auch eine Entität die nur aus einem Token besteht mit einem 'b' kodiert wird und nicht wie im IOB Format mit einem 'i'. Dadurch ergibt sich das folgende Format:

    BOS add clem burke in my playlist Pre-Party R&B Jams EOS o o b-mucic_item i-music_item o b-playlist_owner o b-playlist i-playlist i-playlist
    

Mit dem folgenden Python Code wird der Inhalt der im data-Verzeichnis liegenden Dateien in das vorgestellte Format transformiert. Die Dateien werden dabei im Verzeichnis `data/corpus` und einem Verzeichnis mit dem Titel der Intent-Kategorie abgelegt. Als Dateinamen werden `UUID`s verwendet. 


In [1]:
import os
import json
import re
from shutil import rmtree
import uuid

In [2]:
def clean_formatted(formatted_directory_path):
    if not os.path.exists(formatted_directory_path) or not os.path.isdir(formatted_directory_path):
        print(f'Pfad {formatted_directory_path} ist kein Verzeichnis oder existiert nicht')
        return

    rmtree(formatted_directory_path)
    print('Alte Corpus-Dateien gelöscht')

def convert_file(json_file_path, corpus_root):
    print(f'Try converting file at path {json_file_path}')
    if not os.path.isfile(json_file_path):
        print(f'File {file_path} does not exists', json_file_path)
        return 
    formatted_lines = []
    intent_category = None
    all_slots_set = set()
    all_slots_set.add('o')
    with open(json_file_path, 'r', encoding='latin-1') as json_file:
        json_content = json.load(json_file)
        intent_category = next(iter(json_content))
        for sentence_block in json_content[intent_category]:
            sentence = ""
            slots = []
            for sentence_data_block in sentence_block['data']:
                sentence_part = sentence_data_block['text']
                sentence_part = re.sub('\n', '', sentence_part)
                if sentence_part != '':
                    sentence += sentence_part
                sentence_part_len = len(sentence_part.split())
                if 'entity' in sentence_data_block:
                    entity_type = sentence_data_block['entity']
                    if sentence_part_len > 1:
                        firstSlot = True
                        for i in range(sentence_part_len):
                            if firstSlot:
                                slots.append('b-' + entity_type)
                                firstSlot = False
                                all_slots_set.add('b-' + entity_type)
                            else:
                                slots.append('i-' + entity_type)
                                all_slots_set.add('i-' + entity_type)
                    else:
                        slots.append('b-' + entity_type)
                        all_slots_set.add('b-' + entity_type)
                else:
                    for i in range(sentence_part_len):
                        slots.append('o')
            formatted_lines.append(construct_row(sentence, intent_category, slots))
    print(f'Finished converting file at path {json_file_path}. Writing to file...')
    write_to_file(intent_category, formatted_lines, corpus_root)
    return all_slots_set
    

def construct_row(sentence, intent, slots):
    row = ''
    row += sentence
    row += '#!#'
    row += intent
    row += '#!#'
    row += ' '.join(slots)
    row += '\n'
    return row


def write_to_file(intent, lines, corpus_root):
    if intent is None or intent == '':
        print('No intent')
        return
        
    base_output_directory = corpus_root
    output_file_path = base_output_directory + intent + '/' + str(uuid.uuid4()) + '.txt'

    if not os.path.exists(base_output_directory + intent): 
        os.makedirs(base_output_directory + intent)
    
    with open(output_file_path, 'a') as output_file:
        output_file.writelines(lines)


def write_slots_to_file(corpus_root, slots):
    with open(corpus_root + 'slots.txt', 'w') as slots_file:
        lines = [slot + "\n" for slot in slots]
        slots_file.writelines(lines)


def iterate_over_json_files_in_directory(directory_path, corpus_root):
    if not os.path.exists(directory_path) or directory_path is not os.path.isdir(directory_path):
        print(f"Das Pfad {directory_path} existiert nicht oder ist kein Verzeichnis.")

    slots = set()
    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):  # Nur JSON-Dateien berücksichtigen
            file_path = os.path.join(directory_path, filename)
            slots_from_file = convert_file(file_path, corpus_root)
        slots.update(slots_from_file)

    write_slots_to_file(corpus_root, slots)
    print('Finished converting all files!')
    
    
clean_formatted('data/corpus')
iterate_over_json_files_in_directory('data', 'data/corpus/')

Alte Corpus-Dateien gelöscht
Das Pfad data existiert nicht oder ist kein Verzeichnis.
Try converting file at path data/validate_PlayMusic.json
Finished converting file at path data/validate_PlayMusic.json. Writing to file...
Try converting file at path data/train_SearchCreativeWork_full.json
Finished converting file at path data/train_SearchCreativeWork_full.json. Writing to file...
Try converting file at path data/train_AddToPlaylist_full.json
Finished converting file at path data/train_AddToPlaylist_full.json. Writing to file...
Try converting file at path data/train_RateBook_full.json
Finished converting file at path data/train_RateBook_full.json. Writing to file...
Try converting file at path data/train_SearchScreeningEvent_full.json
Finished converting file at path data/train_SearchScreeningEvent_full.json. Writing to file...
Try converting file at path data/validate_GetWeather.json
Finished converting file at path data/validate_GetWeather.json. Writing to file...
Try converting f

Als nächstes wird geprüft, ob auch alle Einträge in der Textdatei enthalten sind. Dafür werden die Einträge in der train- und validate.json mit der Anzahl der Zeilen in der Textdatei verglichen.

In [3]:
!wc -l data/formatted/RateBook.txt

wc: data/formatted/RateBook.txt: No such file or directory


In [4]:
!jq '.RateBook | length' data/train_RateBook_full.json

1956


In [5]:
!jq '.RateBook | length' data/validate_RateBook.json

100


Man sieht, dass die Anzahl der `data`-Blöcke aus den JSON-Dateien der Anzahl der Zeilen in der erzeugten Textdatei entspricht. Die Konvertierung war also erfolgreich.

In [21]:
!head -n 10 data/formatted/RateBook.txt

BOS rate The Lotus and the Storm zero of 6 EOS RateBook o b-object_name i-object_name i-object_name i-object_name i-object_name b-rating_value o b-best_rating
BOS Rate The Fall-Down Artist 5 stars. EOS RateBook o b-object_name i-object_name i-object_name b-rating_value b-rating_unit o
BOS Rate the current novel one points EOS RateBook o o b-object_select b-object_type b-rating_value b-rating_unit
BOS rate The Ape-Man Within 4 EOS RateBook o b-object_name i-object_name i-object_name b-rating_value
BOS I give The Penalty three stars EOS RateBook o o b-object_name i-object_name b-rating_value b-rating_unit
BOS rate this novel a 4 EOS RateBook o b-object_select b-object_type o b-rating_value
BOS give 5 out of 6 points to Absolutely, Positively Not series EOS RateBook o b-rating_value o o b-best_rating b-rating_unit o b-object_name i-object_name i-object_name b-object_part_of_series_type
BOS I give Emile, or On Education five points. EOS RateBook o o b-object_name i-object_name i-object_nam

Quellen

* 1: https://investors.sonos.com/news-and-events/investor-news/latest-news/2019/Sonos-Announces-Acquisition-of-Snips/default.aspx, [Online, 07.03.2025]
* 2: Coucke A. et al., "Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces." 2018, [Online: https://arxiv.org/abs/1805.10190, 07.03.2025]
* 3: NLTK Team, tree2conlltags, [Online: https://www.nltk.org/_modules/nltk/chunk/util.html#tree2conlltags, 13.03.2025]
* 4: Explosion, spaCy convert, [Online: https://spacy.io/api/cli#converters, 13.03.2025]
* 5: Ramshaw und Marcus, "Text Chunking using Transformation-Based Learning" 1995, [Online: https://arxiv.org/abs/cmp-lg/9505040, 14.03.2025]

## Erstellen eines Corpus

Nachdem nun die Daten in ein verwendbares Format tranformiert wurden, müssen wir den effiziente Zugriff auf die Daten sicherstellen. Dafür bietet die Python-Bibliothek NLTK verschiedene `CorpusReader` Klassen zur Verfügung. Diese Klassen ermöglichen über Methoden den Zugriff auf die Dokumente selbst, sowie auf weitere Dateien eines Corpus, wie zum Beispiel die Lizenz des Corpus oder eine README.md Datei. NLTK bietet eine große Anzahl an CorpusReadern für verschiedenste Szenarien an. Ein einfacher CorpusReader ist der `PlainTextCorpusReader` [7]. Dieser bietet Zugriff auf reine Textdateien. Eine vollständige Liste ist in der [Dokumentation](https://www.nltk.org/api/nltk.corpus.reader.html) von NLTK zu finden.

Für das zuvor definierte Format gibt es keinen passenden vorgefertigten Reader. Daher müssen wir einen eigenen erstellen. Dafür kann die `CorpusReader`-Basisklasse [8] erweitert werden oder ein bestehender verwendet werden. Wir werden den `CategorizedPlaintextCorpusReader` als Basis verwenden. Dieser ist eine Erweiterung des zuvor erwähnten `PlainTextCorpusReader`, welcher Zugriff auf Textdateien bietet.  Durch die Nutzung des `CategorizedPlaintextCorpusReader` können die Dateien zusätzlich in Kategorien unterteilt werden. Dies geschieht über reguläre Ausdrücke. Im nachfolgenden Code definiert der reguläre Ausdruck _category_pattern_ die Kategorie als den Namen des Verzeichnisses, indem die Textdateien liegen.

In [1]:
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
from nltk import wordpunct_tokenize
import codecs

In [2]:
class IOBCorpusReader(CategorizedPlaintextCorpusReader):
    
    def __init__(self, root, fileids, cat_pattern, encoding='utf-8'):
        super().__init__(root, fileids, cat_pattern=cat_pattern, encoding=encoding)
        self.corpus_root = root

    def resolve(self, fileids=None, categories=None):

        if fileids is not None and categories is not None:
            raise ValueError('Specify only one')

        if categories is not None:
            return self.fileids(categories)
        else:
            return fileids

    def docs(self, fileids=None, categories=None):

        fileids = self.resolve(fileids, categories)

        for path, encoding in self.abspaths(fileids, include_encoding=True):
            with codecs.open(path, 'r', encoding=encoding) as file:
                yield file.read()
        
    def lines(self, fileids=None, categories=None):

        for doc in self.docs(fileids, categories):
            for line in doc.split('\n'):
                yield line

    def line_parts(self, fileids=None, categories=None):

        for line in self.lines(fileids, categories):
            if line == '':
                continue
            yield line.split('#!#')

    def intents(self, fileids=None, categories=None):

        for sentence, intent, labels in self.line_parts(fileids, categories):
            yield intent

    def count_intents(self, fileids=None, categories=None):
        
        intents = set(intent for sentence, intent, labels in self.intents(fileids, categories))
        return len(intents)

    def slots(self, fileids=None, categories=None):

        for sentence, intent, slots in self.line_parts(fileids, categories):
            yield slots.split()

    def count_slots(self, fileids=None, categories=None):
        slots = set()
        for entry in self.slots(fileids, categories):
            for token in entry:
                slots.add(token)
        return len(slots)

    def sentences(self, fileids=None, categories=None):

        for sentence, intent, labels in self.line_parts(fileids, categories):
            yield sentence

    def sentences_and_labels(self, fileids=None, categories=None):

        for sentence, intent, slots in self.line_parts(fileids, categories):
            yield sentence, intent, slots

    def all_slots(self):
        
        with open(self.corpus_root + '/slots.txt', 'r') as slots_file:
            return slots_file.read().splitlines()

    
file_pattern = r'.*/[0-9a-f]{8}-[0-9a-f]{4}-4[0-9a-f]{3}-[89ab][0-9a-f]{3}-[0-9a-f]{12}\.txt'
category_pattern = r'([^/]+)/[^/]+\.txt$'
corpus = IOBCorpusReader('data/corpus', file_pattern, cat_pattern=category_pattern)
        

Quellen

* 6: Bengfort, Benjamin, et al. Applied Text Analysis with Python : Enabling Language-Aware Data Products with Machine Learning, O'Reilly Media, Incorporated, 2018. ProQuest Ebook Central, https://ebookcentral.proquest.com/lib/fh-swf/detail.action?docID=5425029.
* 7: NLTK Project, PlainTextCorpusReader, 2024, [Online: https://www.nltk.org/api/nltk.corpus.reader.plaintext.html#nltk.corpus.reader.plaintext.PlaintextCorpusReader, 17.03.2025]
* 8: NLTK project, CorpusReader, 2024, [Online: https://www.nltk.org/api/nltk.corpus.reader#nltk.corpus.reader.CorpusReader, 17.03.2025]

## Modelle

Nachdem nun die Daten vorbereitet wurden und der Zugriff über einen CorpusReader komfortabel möglich ist. Widmen wir uns den Modellen die wir für die Analyse der Infos nutzen wollen.
Zuerst werden zwei getrennte Modelle verwendet. Eines für die Intent Klassifikation und eines für die Bestimmung der Slots. Anschließend wird der neuere Ansatz der gemeinsamen Bestimmung des Intents und der Slots gezeigt. Dieses Ansatz bringt den Vorteil, dass der Text nur einmal verareitet werden muss. Zusätzlich bringt es den Vorteil, dass die Zusammenhänge von Intents und Slots berücksichtigt werden können. So kann der Intent oder die Slots jeweils helfen, das andere Ergebnis zu verbessern. Ein Beispiel wäre die Buchung von einem Flug oder einem Tisch in einem Restaurant. So kann ein Satz zur Buchung mit zwei enthaltenen Ortsangaben besser kategorisiert werden, wenn etwas über den Kontext bekannt ist. Sind zwei Städte in dem Befehl angegeben handelt es sich wahrscheinlich eher um die Buchung eines Fluges anstatt der Buchung in einem Restaurant. Wohingegen die Buchung eines Restaurant eher auf nur eine Stadt hindeutet und die zweite Ortsangabe eventuell aufschluss über einen Stadtteil oder die gewüschte Lage des Tisches gibt.

In [3]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, BertForTokenClassification
from datasets import Dataset
from sklearn.model_selection import train_test_split


Zuerst holen wir die Daten für das Training aus dem Corpus. Dafür werden die Methoden für die einzelnen Bestandteile verwendet. Anschließend verwenden wir die Funktion `train_test_split` aus dem SciKit-Learn Paket um diese in Trainings- und Validierungsteile zu trennen.

In [9]:
sentences = list(corpus.sentences())
intents = list(corpus.intents())
slots = list(corpus.slots())

train_sentences, test_sentences, train_intents, test_intents, train_slots, test_slots = train_test_split(
    sentences, intents, slots, test_size=0.2, random_state=42
)

In den nachfolgenden Beispielen werden Varianten des BERT Modells verwenden. BERT ist ein bidirektionaler encoder-only Transformer. Es wurde im Oktober 2018 von Forschern bei Google vorgestellt. BERT wurde mittels 'masked token prediction', also das Vorhersagen eines maskierten Tokens basierend auf den vorherigen Token, auf dem BookCorpus und dem Englischen Wikipedia trainiert. Durch seine Architektur kann ein bereits trainiertes BERT Modell durch das hinzufügen einer einzigen neuen Schicht für verschiedenste Aufgaben angepasst werden [9].
In den folgenden Beispielen werden die von HuggingFace bereitgestellten Varianten verwendet. Die Transformers Bibliothek von HuggingFace bietet unter anderem folgende Varianten von Bert an: `BertForNextSentencePrediction`, `BertForQuestionAnswering` und, für unser Beispiel relevant, `BertForSequenceClassification` für die Klassifikation von Sätzen und `BertForTokenClassification` für die Klassifizierung von einzelnen Token, in diesem Fall Wörtern, an [10].
BERT ist außerdem in verschiedenen Ausführungen verfügbar die sich in der Anzahl der Parameter und den verwendeten Trainigsdaten unterscheiden. Im weiteren Verlauf wird als Basis das `bert-base-uncased` Modell verwendet. Dieses besitzt 110M Parameter und wurde auf einem 'uncased' Datensatz, nur Kleinbuchstaben, trainiert. Als weitere Ausführungen gibt es noch die 'large' Variante mit 340M Parametern und jeweils eine 'cased' Variante mit Groß- und Kleinschreibung [11].

Quellen:

* 9: Jacob Devlin and Ming-Wei Chang and Kenton Lee and Kristina Toutanova, BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding, 2018, [Online: https://arxiv.org/abs/1810.04805, 23.06.2025]
* 10: HuggingFace Inc., Transformers BERT, 2025, [Online: https://arxiv.org/abs/1810.04805, 23.06.2025]
* 11: HuggingFace Inc., BERT release, 2025, [Online: https://huggingface.co/collections/google/bert-release-64ff5e7a4be99045d1896dbc, 23.06.2025]

Da in den nachfolgenden Beispielen immer ein Tokenizer benötigt wird und dieser nicht Modellunanhängig sein muss, wird er hier zu Anfang ein defineirt.

In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

NameError: name 'BertTokenizer' is not defined

In der folgenden Zelle trainieren wir das `BertForSequenceClassification` Modell für die Klassifikation der Intents aus unseren Daten.
Zuerst wird der `BertTokenizer` verwendet um aus den Sätzen die Encodings zu erstellen. Durch die Parameter padding werden die Sätze auf eine einheitliche Länge gebracht, indem Padding-Token angehangen werden. Sätze die Länger als die vom Modell unterstützte Länge sind werden mit dem Parameter `truncation` gekürzt.
Im zweiten Abschnitt werden die Text-Intents in Nummernlabel umgewandelt.
Anschließend werden die Encodings in `Datasets` umgewandelt dabei werden die erzeugten "input_ids" und die "attention_mask", die die Padding-Token vom eigentlichen Satz unterscheidet, verwendet.
Die vorbereiteten Daten werden dann mit einem HuggingFace `Trainer` für das Training an des BertForSequenceClassification Modell übergeben und das Modell trainiert.

In [10]:
train_encodings = tokenizer(train_sentences, padding=True, truncation=True, return_tensors="pt")
test_encodings = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="pt")

intent_label_mapping = {intent: i for i, intent in enumerate(set(train_intents))}
train_labels = torch.tensor([intent_label_mapping[intent] for intent in train_intents])
test_labels = torch.tensor([intent_label_mapping[intent] for intent in test_intents])

train_dataset = Dataset.from_dict({
    "input_ids": train_encodings["input_ids"],
    "attention_mask": train_encodings["attention_mask"],
    "labels": train_labels
})

test_dataset = Dataset.from_dict({
    "input_ids": test_encodings["input_ids"],
    "attention_mask": test_encodings["attention_mask"],
    "labels": test_labels
})

def encode_examples(examples):
    return {
        "input_ids": torch.tensor(examples["input_ids"]),
        "attention_mask": torch.tensor(examples["attention_mask"]),
        "labels": torch.tensor(examples["labels"])
    }

train_dataset = train_dataset.map(encode_examples, batched=True)
test_dataset = test_dataset.map(encode_examples, batched=True)

model_intent = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(intent_label_mapping))

training_args_intent = TrainingArguments(
    output_dir="./intent_results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    eval_strategy="epoch",
    logging_dir=None,
    report_to=[]
)

trainer_intent = Trainer(
    model=model_intent,
    args=training_args_intent,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer_intent.train()


Map:   0%|          | 0/11587 [00:00<?, ? examples/s]

Map:   0%|          | 0/2897 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.063290
2,No log,0.056072
3,0.205400,0.052450


/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=546, training_loss=0.19020338089038163, metrics={'train_runtime': 130.7018, 'train_samples_per_second': 265.957, 'train_steps_per_second': 4.177, 'total_flos': 732427682531850.0, 'train_loss': 0.19020338089038163, 'epoch': 3.0})

Da jetzt das Modell für die Klassifikation der Sätze zu den Intents fertig ist, behandeln wir im nächsten Schritt das Modell für die Klassifikation der Token.
Da der BertTokenizer Wörter in Subwords unterteilt kann es passieren, dass die IOB-Label nicht mehr zu passen. Um dies zu beheben sorgt die nachfolgende FUnktion dafür, dass bei einem Subword, welche immer mit einem `##` gekenzeichnet sind, ein entsprechenes IOB-Label eingefügt wird.

In [11]:
def align_labels_with_iob_and_collect_set(encoded_sentences, labels, tokenizer):
    aligned_labels = []
    new_labels_set = set()
    
    for input_ids, sentence_labels in zip(encoded_sentences["input_ids"], labels):
        expanded_labels = []
        label_index = 0
        
        for token_id in input_ids:
            token = tokenizer.convert_ids_to_tokens([token_id])[0]
            
            if token.startswith("##"):
                # Subword bekommt ein I-Label (falls vorhanden)
                if sentence_labels[label_index].startswith("B-"):
                    new_label = "I-" + sentence_labels[label_index][2:]
                    expanded_labels.append(new_label)
                else:
                    new_label = sentence_labels[label_index]
                    expanded_labels.append(new_label)
            else:
                # Hauptwörter bekommen das ursprüngliche Label
                new_label = sentence_labels[label_index]
                expanded_labels.append(new_label)
                label_index += 1
            
            # Füge das neue Label zum Set hinzu
            new_labels_set.add(new_label)

            # Sicherheitscheck, um Index-Überläufe zu vermeiden
            if label_index >= len(sentence_labels):
                break

        aligned_labels.append(expanded_labels)
    
    return aligned_labels, new_labels_set


Bei diesem Modell ist das Vorgehen, wie bei dem Vorherigen. Die Label werden mit dem Tokeinzer verarbeitet und danach mit der zuvor definierten Funktion aufbereitet. Die Aufbereitung gibt als Ergebnis zunächst die Label zurück und eine Liste an neu erstellten Label. Dies ist notwendig da durch die Aufteilung in Subwords, zuvor nicht vorkommende I-Label entstanden sein können.
Jetzt wird mit dem Labeln des Corpus und den neuen Labeln wieder ein Mapping auf Zahlen durchgeführt.
Die codierten Label werden dann auf eine gemeinsame Länge gepadded und anschließend wieder in ein `Dataset` überführt und mittels des Trainers für das Training des Modells verwendet.

In [12]:
train_encodings_slots = tokenizer(train_sentences, padding=True, truncation=True, return_tensors="pt")
test_encodings_slots = tokenizer(test_sentences, padding=True, truncation=True, return_tensors="pt")

train_adjusted_labels, train_new_label = align_labels_with_iob_and_collect_set(train_encodings_slots, train_slots, tokenizer)
test_adjusted_labels, test_new_label = align_labels_with_iob_and_collect_set(test_encodings_slots, test_slots, tokenizer)

all_label_corpus = set(corpus.all_slots())
all_label_corpus.update(train_new_label)
all_label_corpus.update(test_new_label)

slot_label_mapping = {slot: i for i, slot in enumerate(all_label_corpus)}
train_slot_labels = [[slot_label_mapping[tag] for tag in tags] for tags in train_slots]
test_slot_labels = [[slot_label_mapping[tag] for tag in tags] for tags in test_slots]

# Hier Fehler weil 0 = I_country??!?!
def pad_labels(labels, max_len):
    return [label + [0] * (max_len - len(label)) for label in labels]

train_max_len = max(len(encoding) for encoding in train_encodings_slots['input_ids'])
train_slot_labels = torch.tensor(pad_labels(train_slot_labels, train_max_len))

test_max_len = max(len(encoding) for encoding in test_encodings_slots['input_ids'])
test_slot_labels = torch.tensor(pad_labels(test_slot_labels, test_max_len))

train_dataset_slots = Dataset.from_dict({
    "input_ids": train_encodings_slots["input_ids"],
    "attention_mask": train_encodings_slots["attention_mask"],
    "labels": train_slot_labels
})

test_dataset_slots = Dataset.from_dict({
    "input_ids": test_encodings_slots["input_ids"],
    "attention_mask": test_encodings_slots["attention_mask"],
    "labels": test_slot_labels
})

def encode_examples_slots(examples):
    return {
        "input_ids": torch.tensor(examples["input_ids"]),
        "attention_mask": torch.tensor(examples["attention_mask"]),
        "labels": torch.tensor(examples["labels"])
    }

train_dataset_slots = train_dataset_slots.map(encode_examples_slots, batched=True)
test_dataset_slots = test_dataset_slots.map(encode_examples_slots, batched=True)

model_slots = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(slot_label_mapping))

training_args_slots = TrainingArguments(
    output_dir="./slot_filling_results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    eval_strategy="epoch",
    logging_dir='./logs',
    report_to=[],
)

trainer_slots = Trainer(
    model=model_slots,
    args=training_args_slots,
    train_dataset=train_dataset_slots, 
    eval_dataset=test_dataset_slots,
)

trainer_slots.train()


Map:   0%|          | 0/11587 [00:00<?, ? examples/s]

Map:   0%|          | 0/2897 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.301149
2,No log,0.171944
3,0.381200,0.121745
4,0.381200,0.097350
5,0.381200,0.090857
6,0.096400,0.076714
7,0.096400,0.071514
8,0.096400,0.069398
9,0.060400,0.065737
10,0.060400,0.065329


/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1820, training_loss=0.1564936297280448, metrics={'train_runtime': 399.1398, 'train_samples_per_second': 290.299, 'train_steps_per_second': 4.56, 'total_flos': 2426016135699360.0, 'train_loss': 0.1564936297280448, 'epoch': 10.0})

In der nachfolgenden Zelle wird eine Funktion definiert, mit der wir auf einen Satz eine getrennte Vorhersage mit den zwei Modellen durchführen können. Danach werden mehrere Sätze angegeben, um die die Modelle zu testen.
Die Funktion erwartet einen Satz, ein Modell für die Intent Detection, eines für das Slot Filling, einen Tokenizer und das Gerät auf dem die Vorhersagen ausgeführt werden sollen.
In der Funktion wird zunächst der Satz codiert und auf das gewählte Gerät verschoben. Danach wird der Satz an beide Modelle übergeben und die Vorhersagen ausgegeben. Die Modelle werden für die Verwendung in den Evaluierungsmodus versetzt und auf das gewählte Gerät, in der Regel die GPU, verschoben.

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Funktion für Intent Detection und Slot Filling
def predict_intent_and_slots(sentence, model_intent, model_slots, tokenizer, device):
    # Eingabe vorbereiten
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Intent-Erkennung
    model_intent.eval()
    model_intent.to(device)
    with torch.no_grad():
        intent_outputs = model_intent(**inputs)
    intent_prediction = torch.argmax(intent_outputs.logits, dim=1).item()
    intent_label = list(intent_label_mapping.keys())[intent_prediction]

    # Slot-Filling
    model_slots.eval()
    model_slots.to(device)
    with torch.no_grad():
        slot_outputs = model_slots(**inputs)
    slot_predictions = torch.argmax(slot_outputs.logits, dim=-1).squeeze().tolist()
    slot_tags = [list(slot_label_mapping.keys())[tag] for tag in slot_predictions]

    print(f"Sentence to inspect: {sentence}")
    print(f"Predicted Intent: {intent_label}")
    print(f"Predicted Slots: {slot_tags}")
    print()

sentences = [
    "Add Jimmy Hendrix to my metal playlist",
    "Book a table in the finest restaurant in Paris",
    "I want to book a flight to New York tomorrow morning.",
    "What's the weather in DC?",
    "How is the weather in DC?"
]

for sentence in sentences:
    predict_intent_and_slots(
        sentence, 
        model_intent, 
        model_slots, 
        tokenizer, 
        device
    )


Sentence to inspect: Add Jimmy Hendrix to my metal playlist
Predicted Intent: AddToPlaylist
Predicted Slots: ['o', 'b-artist', 'i-artist', 'o', 'b-playlist_owner', 'b-playlist', 'o', 'i-country', 'i-country', 'i-country']

Sentence to inspect: Book a table in the finest restaurant in Paris
Predicted Intent: BookRestaurant
Predicted Slots: ['o', 'o', 'o', 'o', 'o', 'b-sort', 'b-restaurant_type', 'o', 'b-state', 'i-country', 'i-country']

Sentence to inspect: I want to book a flight to New York tomorrow morning.
Predicted Intent: BookRestaurant
Predicted Slots: ['o', 'o', 'o', 'o', 'o', 'o', 'o', 'b-state', 'i-city', 'o', 'i-timeRange', 'o', 'i-country', 'i-country']

Sentence to inspect: What's the weather in DC?
Predicted Intent: GetWeather
Predicted Slots: ['o', 'o', 'o', 'o', 'b-state', 'o', 'i-country', 'i-country', 'i-country', 'i-country']

Sentence to inspect: How is the weather in DC?
Predicted Intent: GetWeather
Predicted Slots: ['o', 'o', 'o', 'o', 'o', 'b-state', 'o', 'i-coun

Nachdem wir zuvor zwei getrennte Modelle für die Vorhersage verwendet haben, wird dies im nächsten Abschnitt von einem Modell gemacht. Diese herangehensweise nennt sich 'Joint' Intent detection und Slot filling, da hier die beiden Vorhersagen vereint (joint) sind.
Dafür wird wieder ein BERT Modell verwendet. Für diese Aufgabe bietet HuggingFace kein fertiges Modell an. Daher wird für dieses Beispiel die Implementierung eines Joint BERT Modells von [Jang Won Park](https://github.com/monologg) verwendet. Die Implementierung des Modells stammt aus seinem GitHub [Repository](https://github.com/monologg/JointBERT) [11] und basiert auf einem Paper von .....

Quellen:

* 11: Jang Won Park, JointBERT, 2020, [Online: https://github.com/monologg/JointBERT, 25.06.2025]

In [10]:
import sys
import torch.nn as nn
import torch
from transformers import BertPreTrainedModel, BertModel, BertConfig
try:
    from torchcrf import CRF
except ModuleNotFoundError:
    !{sys.executable} -m pip install pytorch-crf
    from torchcrf import CRF

In [11]:
class IntentClassifier(nn.Module):
    def __init__(self, input_dim, num_intent_labels, dropout_rate=0.):
        super(IntentClassifier, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, num_intent_labels)

    def forward(self, x):
        x = self.dropout(x)
        return self.linear(x)


class SlotClassifier(nn.Module):
    def __init__(self, input_dim, num_slot_labels, dropout_rate=0.):
        super(SlotClassifier, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(input_dim, num_slot_labels)

    def forward(self, x):
        x = self.dropout(x)
        return self.linear(x)

In [12]:
class JointBERT(BertPreTrainedModel):
    def __init__(self, config, args, intent_label_lst, slot_label_lst):
        super(JointBERT, self).__init__(config)
        self.args = args
        self.num_intent_labels = len(intent_label_lst)
        self.num_slot_labels = len(slot_label_lst)
        self.bert = BertModel(config=config)  # Load pretrained bert

        self.intent_classifier = IntentClassifier(config.hidden_size, self.num_intent_labels, args.dropout_rate)
        self.slot_classifier = SlotClassifier(config.hidden_size, self.num_slot_labels, args.dropout_rate)

        if args.use_crf:
            self.crf = CRF(num_tags=self.num_slot_labels, batch_first=True)

    def forward(self, input_ids, attention_mask, token_type_ids, intent_label_ids, slot_labels_ids):
        outputs = self.bert(input_ids, attention_mask=attention_mask,
                            token_type_ids=token_type_ids)  # sequence_output, pooled_output, (hidden_states), (attentions)
        sequence_output = outputs[0]
        pooled_output = outputs[1]  # [CLS]

        intent_logits = self.intent_classifier(pooled_output)
        slot_logits = self.slot_classifier(sequence_output)

        total_loss = 0
        # 1. Intent Softmax
        if intent_label_ids is not None:
            if self.num_intent_labels == 1:
                intent_loss_fct = nn.MSELoss()
                intent_loss = intent_loss_fct(intent_logits.view(-1), intent_label_ids.view(-1))
            else:
                intent_loss_fct = nn.CrossEntropyLoss()
                intent_loss = intent_loss_fct(intent_logits.view(-1, self.num_intent_labels), intent_label_ids.view(-1))
            total_loss += intent_loss

        # 2. Slot Softmax
        if slot_labels_ids is not None:
            if self.args.use_crf:
                slot_loss = self.crf(slot_logits, slot_labels_ids, mask=attention_mask.byte(), reduction='mean')
                slot_loss = -1 * slot_loss  # negative log-likelihood
            else:
                slot_loss_fct = nn.CrossEntropyLoss(ignore_index=self.args.ignore_index)
                # Only keep active parts of the loss
                if attention_mask is not None:
                    active_loss = attention_mask.view(-1) == 1
                    active_logits = slot_logits.view(-1, self.num_slot_labels)[active_loss]
                    active_labels = slot_labels_ids.view(-1)[active_loss]
                    slot_loss = slot_loss_fct(active_logits, active_labels)
                else:
                    slot_loss = slot_loss_fct(slot_logits.view(-1, self.num_slot_labels), slot_labels_ids.view(-1))
            total_loss += self.args.slot_loss_coef * slot_loss

        outputs = ((intent_logits, slot_logits),) + outputs[2:]  # add hidden states and attention if they are here

        outputs = (total_loss,) + outputs

        return outputs

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "bert-base-uncased"
joint_config = BertConfig.from_pretrained(model_name)
model_joint = JointBERT.from_pretrained(model_name, config=joint_config, intent_label_lst=dsjnfnsdf, slot_label_lst=jdsifj)
model_joint.to(device)

training_args_joint = TrainingArguments(
    output_dir="./joint_results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    eval_strategy="epoch",
    logging_dir='./logs',
    report_to=[],
)

data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=None)

trainer_joint = Trainer(
    model=model_joint,
    args=training_args_joint,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=, 
    eval_dataset=,
)

trainer_joint.train()

https://arxiv.org/abs/1902.10909

## Zusammenfassung